# Create land cover mask for North America

In [1]:
import rioxarray as rxr
import xarray as xr


In [2]:
modis_path = "../data/raw/MCD12C1v061/MCD12C1.A2021001.061.2022217040006.hdf"
ds = rxr.open_rasterio(modis_path)

ds


<xarray.Dataset>
Dimensions:                                (band: 17, x: 7200, y: 3600)
Coordinates:
  * band                                   (band) int64 1 2 3 4 ... 14 15 16 17
  * x                                      (x) float64 -180.0 -179.9 ... 180.0
  * y                                      (y) float64 89.97 89.92 ... -89.98
    spatial_ref                            int64 0
Data variables:
    Majority_Land_Cover_Type_1             (band, y, x) float64 0.0 0.0 ... nan
    Majority_Land_Cover_Type_1_Assessment  (band, y, x) float64 0.0 0.0 ... nan
    Land_Cover_Type_1_Percent              (band, y, x) uint8 ...
    Majority_Land_Cover_Type_2             (band, y, x) float64 0.0 0.0 ... nan
    Majority_Land_Cover_Type_2_Assessment  (band, y, x) float64 255.0 ... nan
    Land_Cover_Type_2_Percent              (band, y, x) float64 100.0 ... nan
    Majority_Land_Cover_Type_3             (band, y, x) float64 0.0 0.0 ... nan
    Majority_Land_Cover_Type_3_Assessment  (band, y, x) float64 255.0 ... nan
    Land_Cover_Type_3_Percent              (band, y, x) float64 100.0 ... nan
Attributes: (12/55)
    ALGORITHMPACKAGEACCEPTANCEDATE:     2001-01-01
    ALGORITHMPACKAGEMATURITYCODE:       LAUNCH
    ALGORITHMPACKAGENAME:               MOD12C1
    ALGORITHMPACKAGEVERSION:            V2.0
    ASSOCIATEDINSTRUMENTSHORTNAME.1:    MODIS
    ASSOCIATEDINSTRUMENTSHORTNAME.2:    MODIS
    ...                                 ...
    SCIENCEQUALITYFLAGEXPLANATION.1:    See http://landweb.nascom/nasa.gov/cg...
    SHORTNAME:                          MCD12C1
    SOUTHBOUNDINGCOORDINATE:            -90.0
    SPSOPARAMETERS:                     2669
    VERSIONID:                          61
    WESTBOUNDINGCOORDINATE:             -180.0

In [3]:
df = (
    ds["Majority_Land_Cover_Type_1"]  # IGBP classification scheme
    .sel(band=1, x=slice(-125, -65), y=slice(58, 22))
    .drop(["band", "spatial_ref"])
    .to_dataframe()
    .reset_index()
    .rename(columns={"y": "lat", "x": "lon", "Majority_Land_Cover_Type_1": "land_cover"})
)
df["land_cover"] = df["land_cover"].where(df["land_cover"] == 0, other=1)
ds_landcover = df.set_index(["lat", "lon"]).to_xarray()
ds_landcover


<xarray.Dataset>
Dimensions:     (lat: 720, lon: 1200)
Coordinates:
  * lat         (lat) float64 22.02 22.07 22.12 22.17 ... 57.87 57.92 57.97
  * lon         (lon) float64 -125.0 -124.9 -124.9 ... -65.12 -65.07 -65.02
Data variables:
    land_cover  (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0

In [4]:
ds_landcover.to_netcdf("../data/production/land_cover_north_america.nc4", format="NETCDF4")
ds_landcover.close()
